<a href="https://colab.research.google.com/github/brunostre/imp/blob/main/implementacao_simplex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tarefa 8 - Projeto Computacional
===

--------

Bruno Stevanato Trevisan 168170

Renata Trevisan Watanabe 186543

"Ohana quer dizer família. Família quer dizer nunca abandonar ou esquecer"

--------

Fazer uma implementação computacional do algoritmo do método primal simplex para resolver um problema de programação linear (PPL) na forma padrão:

minimizar $f(x) = c^t x$

sujeito a: $Ax = b$

$\quad\quad\quad\quad  x \geq 0$,

em que: $A \in \mathbb{R}^{m\times n}$, $b \in \mathbb{R}^m, c \in \mathbb{R}^n, x \in \mathbb{R}^n$

# **Programa**

**Instruções para uso:**

Dado o PLL em sua forma padrão, para sua resolução:



1.   Não deve conter variaveis artificiais
2.   Sua submição deve ser feito por meio de um arquivo .csv, seguindo os padrões apresentados abaixo.
    1.   Celulas vazias não serão tratadas como zero.
    2.   O vetor dos custos deve ser a primeira linha.
    3.   A matriz dos coeficientes das restrições deve ser a segunda linha, com o seu respectivo valor dos recursos ao final dela.
    4. O separador padrão é a virgula e para o marcador de decimal é o ponto. Caso seu arquivo tenha outro padrão não há problema apenas informar quando perguntado.
    5. Tomamos o problema base para o número de variaveis auxiliares iguais a m. Caso não se encaixe nesse caso indicar qual a base inicial preenchendo a linha de baixo da matriz com um x nas colunas da base e o restante com zeros. 


<figure>
    <img src = https://drive.google.com/uc?id=1wrMiTsZVm4TzG6BVSBPAVKI-FdA-wK46>
    <figcaption align = "center"> Fig.1 - Padrão de formatação do csv </figcapition>
</figure>

Exemplo numérico

<figure>
    <img src = https://drive.google.com/uc?id=1CGkt1svzrEHhWuxTrhBMSrFCaHnCr6lx>
    <figcaption align = "center"> Fig.2 - exemplo </figcapition>
</figure>

## **Código**


**Bibliotecas usadas**

```python
from google.colab import files # Para subimissão do arquivo de entrada
from sympy import *            # Utilizado para imprimir as varivaeis em formato latex 
from numpy import *            # Realizar operações matematicas
import pandas as pd            # Trata a entrada do arquivo csv e transforma nas respectivas variaveis do PL
```

PPL exemplo para explicação do código.

\begin{equation*}
\mathbf{A}=
\begin{pmatrix}
    -3 & \;4 & 1 & 0 & 0\\
    \;\;1 & -1 & 0 & 1 & 0\\
    \;\;1 & \;\;1 & 0 & 0 & 1\\
\end{pmatrix}
\end{equation*}

Ou seja

\begin{equation*}
\mathbf{A}=
\begin{pmatrix}
    a_0 & a_1 & a_2 & a_3 & a_4
\end{pmatrix}
\end{equation*}

onde $a_i$ em python é representado por:


```
A[:,i]
```


### **Pseudo fase I:**

**Definição das matrizes B e N (encontrar uma base factivel)**

Encontrar as variaveis tal que $c_i = 0$

```python
def custos_zero(c):

    indices_custo_zero = []

    for i in range(len(c)):
        if c[i] == 0:
            indices_custo_zero.append(i)

    return indices_custo_zero
```

Logo custos_zero nos retorna:

 ```indices_custo_zero = [2, 3, 4]```

e caimos no **caso I**

**Caso I:** `n > m` e `len(indices_custos_zero) == m` 

```python
def gera_indices(indices_B, indices_N, indices_custo_zero, m, c):

    if len(indices_custo_zero) == m:
        indices_N = array([(i) for i in range(len(c)) if i not in indices_custo_zero])
        indices_B = array([(i) for i in range(len(c)) if i not in indices_N])
        return indices_N, indices_B
```

nos retornando `indices_B = [2, 3, 4] ` e `indices_N = [0, 1]`

para esse caso $x_b$ é factivel pois $x_n = 0$

**Caso II:** `n > m` e `len(indices_custos_zero) < m`

Para esse caso a base sera dada pelo usuario e sua entrada é dada por:

```python
if len(indices_custo_zero) < m:
    entrada = pd.read_csv(Arquivo, sep = Separador, dec = Decimal)
    indices_B = entrada.loc[m+1].to_numpy()[0:n-1]
    indices = []
    try:
        for i in indices_B:
            if i == 'x':
                indices.append(list(indices_B).index(i))
        
        indices_custo_zero = indices
    
    except:
        print('verifique se a base foi inserida no arquivo csv')
```



com B e N definidos constroi B e N a partir dos indices

```python
def constroi_B(B_indices, A, c):
    m,n = shape(A)
    B = zeros((m,m))
    C_B = zeros ((1,m))[0]
    for i in range(m):
        B[:,i] = A[:,B_indices[i]]
        C_B[i] = c[B_indices[i]]

    return B, C_B

def constroi_N(N_indices, A, c):
    m,n = shape(A)
    N = zeros((m,n-m))
    C_n = zeros ((1,n-m))[0]
    for i in range(n-m):
        N[:,i] = A[:,N_indices[i]]
        C_n[i] = c[N_indices[i]]

    return N, C_n
```

### **Fase II:**

#### **Passo 1:** Cálculo da solução basica

para $x_n = 0$

Para calcular $X_b$ resolvemos o seguinte sitema linear:
$$BX_b=b$$

```python
def X_b(B, b):
    try:  
        xb = linalg.solve(B,b)
        return xb, False
    except Exception as e:
        print (e)
        return zeros((shape(b))), 'Problema ilimitado'
```

#### **Passo 2:**

##### 2.1 Calculo do vetor multiplicador simplex

Resolvemos o sistema linear: $$B^t\lambda = c_B$$

```python
def lamb(B, C_B):

    vetor_simplex = linalg.solve(transpose(B), C_B)
    return vetor_simplex
```

##### 2.2 Custos relativos 
O custo relativo é calculado da seguinte forma:
$$\hat{c_i} = c_i - \lambda^ta_i$$


```python
def custos_relativos(C_n, N, vetor_simplex, n, m):

    c_n = zeros((1,n-m))[0]
    for i in range(n-m):
        c_n[i] = C_n[i] - dot(transpose(vetor_simplex), N[:,i])

    return c_n
```

#### 2.3 Escolha da variavel a entrar na base

Para entrar na base a varíavel deve ter custo menor que zero. Se houver mais que uma escolhemos aquela com menor valor.  
```python
def variavel_a_entrar(c_n, indices_N):

    indice_do_que_entra = list(c_n).index(min(c_n))
    otimo = False
    return indices_N[indice_do_que_entra]
```



#### **Passo 3:** Teste de otimalidade

Caso não tenha nenhum custo menor que zero estamos na solução ótima e o programa acaba. Caso contrário continuamos as iterações.
```python
def otimo(c_n):

    if min(c_n) >= 0:
        return True
    else:
        return False
```

#### **Passo 4:** Calculo da direcao simplex
No cálculo da direção simplex fazemos: 

$$By=a_k,$$ onde $a_k$ é a coluna da varíavel que entrou na base.
```python
def direcao_simplex(B, N, indice_do_que_entra, indices_N):

    y = linalg.solve(B, N[:,list(indices_N).index(indice_do_que_entra)])

    return y
```

#### **Passo 5:** Determinar passo e variavel a sair da base

Calculamos por fim o passo fazendo:
$$\hat{\epsilon} = min\{\frac{\hat{x_{Bi}}}{y_i}\},$$ para $y_i > 0$. Caso não exista $y_i>0$ então as soluções serão sempre factíveis para todo $\epsilon > 0$. Dessa forma o problema possui solução ótima ilimitada.

Com o passo cálculado a váriavel a sair da base é aquela utilizada para calcular o passo. 



```python
def passo(x_b, y, n, m, indices_B):

    epsilon = zeros((1,m))[0]

    if max(y) <= 0:
        return 'sem solução'
    else:
        for i in range(m):
            if y[i] == 0:
                epsilon[i] = -1
            else:
                epsilon[i] = x_b[i]/y[i]

    indice_do_que_sai = list(epsilon).index(min([(i) for i in epsilon if i > 0]))
    indice_do_que_sai = indices_B[indice_do_que_sai]

    return False, indice_do_que_sai

def atualiza_BN(A, B, N, c, C_B, C_n, indices_B, indices_N, indice_do_que_entra, indice_do_que_sai):

    pos_que_sai = list(indices_B).index(indice_do_que_sai)
    indices_B[pos_que_sai] = indice_do_que_entra
    C_B[pos_que_sai] = c[indice_do_que_entra]
    pos_que_entra = list(indices_N).index(indice_do_que_entra)
    C_n[pos_que_entra] = c[indice_do_que_sai]
    indices_N[pos_que_entra] = indice_do_que_sai
    B[:,pos_que_sai] = N[:,pos_que_entra]
    N[:,pos_que_entra] = A[:,indice_do_que_sai]

    return B, N, C_B, C_n, indices_B, indices_N
```

# Programa

In [ ]:
#@title #Funções

from google.colab import files 
from sympy import *            
from numpy import * 
import pandas as pd    

def csv_para_array(arquivo,**kwargs):

    separador = ','
    dec = '.'
    enc = None
    for key, value in kwargs.items():
        if 'sep' in key:
            separador = value[key.index('sep')]
        if 'dec' in key:
            dec = value[key.index('dec')]
        if 'enc' in key:
            enc = value[key.index('enc')]

    try:
        entrada = pd.read_csv( arquivo, sep = separador, decimal = dec, encoding = enc, header = None)
        entrada
        m,n = shape(entrada) #m_verdadeiro = m - 1, e o mesmo vale para n
        c = entrada.loc[0].to_numpy()[0:n-1]
        b = entrada.loc[:,n-1].to_numpy()[1:]
        a = entrada.loc[1:m,0:n-2].to_numpy()
        indices_base = entrada.loc[0].to_numpy()[0:n-1]

        print_latex(a, b, c)
        return a, b, c, n-1, m-1, indices_base
    except Exception as e:
        print(e)
        return




def valor_funcao(c, x_base):
    return dot(transpose(c),x_base)[0]




def custos_zero(c):

    indices_custo_zero = []

    for i in range(len(c)):
        if c[i] == 0:
            indices_custo_zero.append(i)

    return indices_custo_zero




def gera_indices(indices_B, indices_N, indices_custo_zero, m, c):

    if len(indices_custo_zero) == m:
        indices_N = array([(i) for i in range(len(c)) if i not in indices_custo_zero])
        indices_B = array([(i) for i in range(len(c)) if i not in indices_N])
        return indices_N, indices_B




def constroi_B(B_indices, A, c):
    m,n = shape(A)
    B = zeros((m,m))
    C_B = zeros ((1,m))[0]
    for i in range(m):
        B[:,i] = A[:,B_indices[i]]
        C_B[i] = c[B_indices[i]]

    return B, C_B




def constroi_N(N_indices, A, c):
    m,n = shape(A)
    N = zeros((m,n-m))
    C_n = zeros ((1,n-m))[0]
    for i in range(n-m):
        N[:,i] = A[:,N_indices[i]]
        C_n[i] = c[N_indices[i]]

    return N, C_n




def constroi_BN_CBCN(B_indices, N_indices, A, c):
    B, C_B = constroi_B(B_indices, A, c)
    N, C_n = constroi_N(N_indices, A, c)
    return B, N, C_B, C_n




def X_b(B, b): #tratar para matrizes singulares
    try:  
        xb = linalg.solve(B,b)
        return xb, False
    except Exception as e:
        print (e)
        return zeros((shape(b))), 'Problema ilimitado'




def lamb(B, C_B):

    vetor_simplex = linalg.solve(transpose(B), C_B)
    return vetor_simplex




def custos_relativos(C_n, N, vetor_simplex, n, m):

    c_n = zeros((1,n-m))[0]
    for i in range(n-m):
        c_n[i] = C_n[i] - dot(transpose(vetor_simplex), N[:,i])

    return c_n




def variavel_a_entrar(c_n, indices_N):

    indice_do_que_entra = list(c_n).index(min(c_n))
    otimo = False
    return indices_N[indice_do_que_entra]




def otimo(c_n):

    if min(c_n) >= 0:
        otimo = True
        return otimo

    else:
        return False




def direcao_simplex(B, N, indice_do_que_entra, indices_N):

    y = linalg.solve(B, N[:,list(indices_N).index(indice_do_que_entra)])

    return y




def passo(x_b, y, n, m, indices_B):

    epsilon = zeros((1,m))[0]

    if max(y) <= 0:
        return 'sem solução', epsilon
    else:
        for i in range(m):
            if y[i] == 0:
                epsilon[i] = -1
            else:
                epsilon[i] = x_b[i]/y[i]

    indice_do_que_sai = list(epsilon).index(min([(i) for i in epsilon if i > 0]))
    indice_do_que_sai = indices_B[indice_do_que_sai]
    otimal = False
    return otimal, indice_do_que_sai




def atualiza_BN(A, B, N, c, C_B, C_n, indices_B, indices_N, indice_do_que_entra, indice_do_que_sai):

    pos_que_sai = list(indices_B).index(indice_do_que_sai)
    indices_B[pos_que_sai] = indice_do_que_entra
    C_B[pos_que_sai] = c[indice_do_que_entra]
    pos_que_entra = list(indices_N).index(indice_do_que_entra)
    C_n[pos_que_entra] = c[indice_do_que_sai]
    indices_N[pos_que_entra] = indice_do_que_sai
    B[:,pos_que_sai] = N[:,pos_que_entra]
    N[:,pos_que_entra] = A[:,indice_do_que_sai]

    return B, N, C_B, C_n, indices_B, indices_N




def print_latex(A, b, c):
    init_printing()
    display(Eq(S('A'),Matrix(A), evaluate = False))
    print()
    display(Eq(S('b'),Matrix(b), evaluate = False))
    print()
    display(Eq(S('c'),Matrix(c), evaluate = False))
    print()
    print()



In [ ]:
#@title # Entrada

#@markdown ### Insira o nome do arquivo:
#uploaded = files.upload()
Arquivo = '.csv' #@param {type:"string"}
Separador = ";" #@param [] {allow-input: true}
Decimal = "," #@param [""] {allow-input: true}


In [ ]:
#@title # Main

uploaded = files.upload()

indices_N = []
indices_B = []
otimal = False

A, b, c, n, m, indices_base = csv_para_array(Arquivo, sep = Separador, dec = Decimal)

indices_custo_zero = custos_zero(c)

if len(indices_custo_zero) < m:
    entrada = pd.read_csv(Arquivo, sep = Separador, dec = Decimal)
    indices_B = entrada.loc[m+1].to_numpy()[0:n-1]
    indices = []
    try:
        for i in indices_B:
            if i == 'x':
                indices.append(list(indices_B).index(i))
        
        indices_custo_zero = indices
    
    except:
        print('verifique se a base foi inserida no arquivo csv')

indices_N, indices_B = gera_indices(indices_B, indices_N, indices_custo_zero, m, c)

B, N, C_B, C_n = constroi_BN_CBCN(indices_B, indices_N, A, c)

t = 0

while (otimal == False) and (t < 10):

    x_b, otimal = X_b(B, b)
    if otimal != False:
        if otimal != True:
            print(otimal)
        break
    vetor_simplex = lamb(B, C_B)
    c_n  = custos_relativos(C_n, N, vetor_simplex, n, m)
    indice_do_que_entra = variavel_a_entrar(c_n, indices_N)
    otimal = otimo(c_n)
    if otimal != False:
        if otimal != True:
            print(otimal)
        break
    y = direcao_simplex(B, N, indice_do_que_entra, indices_N)
    otimal, indice_do_que_sai = passo(x_b, y, n ,m , indices_B)
    if otimal != False:
        if otimal != True:
            print(otimal)
        break
    B, N, C_B, C_n, indices_B, indices_N = atualiza_BN(A, B, N, c, C_B, C_n, indices_B, indices_N, indice_do_que_entra, indice_do_que_sai)
    t = t + 1

x_ot = zeros((n,1))
x_otimo = zeros((n,1))
for j in range(1,n+1):
    x_ot[j-1,:] = j
for i in indices_B:
    x_otimo[i,:] = x_b[list(indices_B).index(i)]
if otimal == True:
    init_printing()
    display(Eq(S("x"),Matrix(x_otimo), evaluate = False))
    vf = valor_funcao(c,x_otimo)
    print()
    display(Eq(S("f(x)"),vf, evaluate = False))


Obs.: Para evitar problemas reiniciar o ambiente de execução do programa a cada teste rodado. (CTRL + M + . )